University of Zagreb\
Faculty of Electrical Engineering and Computing

## Text Analysis and Retrieval 2021/2022
https://www.fer.unizg.hr/predmet/apt/

------------------------------

### Neural NLP
### LAB 3


*Version: 1.0*

(c) 2022 Josip Jukić, Jan Šnajder

Submission deadline: **May 29, 2022, 23:59 CET** 

------------------------------

### Instructions

Hello visitor, this lab assignment consists of three parts. Your task boils down to filling out the missing parts of code and evaluating the cells. These parts are indicated by the "YOUR CODE HERE" template.

Each subtask is supplemented by several tests that you can run. Apart from that, there are additional test that will be executed after submition. If your solution is valid and it passes all of the visible tests, there shouldn't be any problems with the additional tests.

**IMPORTANT: Don't change the names of the predefined methods or random seeds**, because the tests won't execute properly.

You're required to do this assignment **on your own**.

If you stumble upon problems, please refer to josip.jukic@fer.hr for office hours.

## Tasks

### 1. Machine Translation

Machine Translation (MT) is the task of automatically converting one natural language into another, preserving the meaning of the input text, and producing fluent text in the output language. While machine translation is one of the oldest subfields of artificial intelligence research, the recent shift towards large-scale empirical techniques has led to very significant improvements in translation quality.

In this lab assignment, we will use pre-trained sequence-to-sequence models for machine translation. Additionally, we will consider two text generation strategies: greedy decoder and beam search.

#### (a)

Assuming that we are in possesion of a pre-trained model, we still need to figure out how exactly are we going to generate tokens. One of the simplest approaches is to retrieve the most probable token in each step.
Implement `greedy_decoder` for language generation. Greedy method retrieves the index of the most probable token for each timestep in a sequence.

In [1]:
import numpy as np


def greedy_decoder(array):
    return np.argmax(array, axis=1)

In [2]:
ex1a1 = np.array(
    [
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.5, 0.4],
        [0.2, 0.1, 0.3, 0.4, 0.5],
        [0.3, 0.4, 0.5, 0.2, 0.1],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.5, 0.3, 0.4, 0.2],
        [0.5, 0.4, 0.3, 0.2, 0.1],
    ]
)

sol1a1 = np.array([4, 0, 4, 0, 3, 4, 2, 0, 1, 0])

assert (greedy_decoder(ex1a1) == sol1a1).all()

#### (b)

Greedy approach has the benefit that it is very fast, but the quality of the final output sequences may be far from optimal. Instead of greedily choosing the most likely next step as the sequence is constructed, the beam search expands all possible next steps and keeps the `k` most likely, where `k` is a parameter and controls the number of beams or parallel searches through the sequence of probabilities.

The local beam search algorithm keeps track of `k` states rather than just one. It begins with `k` randomly generated states. At each step, all the successors of all `k` states are generated. If any one is a goal, the algorithm halts. Otherwise, it selects the k best successors from the complete list and repeats. This is illustrated in the figure below.

Implement `beam_search_decoder`.

![Beam search](img/beamsearch.jpg)

In [3]:
def k_argmax(arr, k):
    ret = []
    tmp = arr.copy()
    while len(ret) != k:
        curr_max = np.argmax(tmp)
        tmp[curr_max] = -1
        ret.append(curr_max)
    return ret


def beam_search_decoder(data, k):
    candidates = [[]]
    for idx, row in enumerate(data):
        maxes = k_argmax(row, k)
        
        new_candidates = []
        for candidate in candidates:
            for max in maxes:
                tmp_candidate = candidate.copy()
                tmp_candidate.append(max)
                new_candidates.append(tmp_candidate.copy())
                tmp_candidate.remove(max)
            
        candidates = new_candidates[:k]
    return candidates

In [4]:
ex1b1 = np.array(
    [
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.5, 0.4],
        [0.2, 0.1, 0.3, 0.4, 0.5],
        [0.3, 0.4, 0.5, 0.2, 0.1],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.5, 0.3, 0.4, 0.2],
        [0.5, 0.4, 0.3, 0.2, 0.1],
    ]
)

sol1b1 = np.array(
    [
        [4, 0, 4, 0, 3, 4, 2, 0, 1, 0],
        [4, 0, 4, 0, 3, 4, 2, 0, 1, 1],
        [4, 0, 4, 0, 3, 4, 2, 0, 1, 2],
    ]
)

assert (beam_search_decoder(ex1b1, 3) == sol1b1).all()

#### (c)

Finally, let's employ a pre-trained sequence-to-sequence transformer from the [`hugggingface`](https://huggingface.co/) library. Specifically, we are going to use `mBART-large-50`, which can be employed in the [zero-shot](https://en.wikipedia.org/wiki/Zero-shot_learning) setup.

In [5]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


# Load the model and its corresponding tokenizer.
model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/mbart-large-50-many-to-many-mmt"
)
tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

Take a look at the example below to see how to use the pre-trained model in the zero-shot setup. We are translating from Finnish (fi_FI) to English (en_XX).

In [6]:
fi_text = "Aamu kuluu aatellessa, päivä päätä käännellessä."
tokenizer.src_lang = "fi_FI"
encoded_ar = tokenizer(fi_text, return_tensors="pt")
generated_tokens = model.generate(
    **encoded_ar, max_length=50, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
)
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

'The morning goes by in bed, and the day ends in reverse.'

We can control the decoding algorithm by setting the `num_beams` parameter. If it is set to `None`, tokens will be decoded greedily. Additionally, we can control the sequence max length with `max_length` and early stopping with `early_stopping`. When set to True, `early_stopping` indicates that generation is finished when all beam hypotheses reached the EOS (end-of-sequence) token.

Implement the `translate` method which generalizes translation using `mbart` to any source and target language supported by the model. If in doubt, refer to the previous example. Be sure to use all of the arguments, most of which are going to be forwarded to the model's `generate` method. `batch_decode` wraps the decoded tokens into a list, so don't forget to extract the first element from the list to get the actual string.

In [12]:
def translate(
    model,
    tokenizer,
    text,
    src_lang,
    tgt_lang="en_XX",
    num_beams=None,
    max_length=50,
    early_stopping=True,
):
    tokenizer.src_lang = src_lang
    encoded_ar = tokenizer(text, return_tensors="pt")
    
    if num_beams == None:
        early_stopping = False
    
    generated_tokens = model.generate(
        **encoded_ar, max_length=max_length, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang], num_beams=num_beams, early_stopping=early_stopping)
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

Translate Finnish (fi_FI) and Portugese (pt_PT) texts to English (see the cell below) using the `translate` method. Is greedy decoding doing OK or is it better to use beam search? If beam search is better, which `k` seems to perform well?

In [13]:
fi_text_1 = "Ahneus vie linnut taivaalta ja kalat merestä."
fi_text_2 = "Aika on rahaa, sanoi työtön kun kellonsa myi."

pt_text_1 = "Mais vale um pássaro na mão do que dois voando."
pt_text_2 = "Diz-me com quem andas e eu te direi quem és."

In [14]:
ex1c1 = "Anfangen ist leicht, Beharren ist Kunst"
assert translate(model, tokenizer, ex1c1, "de_DE") == "Starting is easy, barking is art"

In [ ]:
print(f'Sentence #1: {fi_text_1}')
print(f'\tGreedy: {translate(model, tokenizer, fi_text_1, "fi_FI")}')
print(f'\tBeam serach (k=2): {translate(model, tokenizer, fi_text_1, "fi_FI", num_beams=2)}')
print(f'\tBeam serach (k=3): {translate(model, tokenizer, fi_text_1, "fi_FI", num_beams=3)}')
print(f'\tBeam serach (k=5): {translate(model, tokenizer, fi_text_1, "fi_FI", num_beams=5)}')
print(f'\tBeam serach (k=20): {translate(model, tokenizer, fi_text_1, "fi_FI", num_beams=20)}')
print()

print(f'Sentence #2: {fi_text_2}')
print(f'\tGreedy: {translate(model, tokenizer, fi_text_2, "fi_FI")}')
print(f'\tBeam serach (k=2): {translate(model, tokenizer, fi_text_2, "fi_FI", num_beams=2)}')
print(f'\tBeam serach (k=3): {translate(model, tokenizer, fi_text_2, "fi_FI", num_beams=3)}')
print(f'\tBeam serach (k=5): {translate(model, tokenizer, fi_text_2, "fi_FI", num_beams=5)}')
print(f'\tBeam serach (k=20): {translate(model, tokenizer, fi_text_2, "fi_FI", num_beams=20)}')
print()

print(f'Sentence #3: {pt_text_1}')
print(f'\tGreedy: {translate(model, tokenizer, pt_text_1, "pt_PT")}')
print(f'\tBeam serach (k=2): {translate(model, tokenizer, pt_text_1, "pt_PT", num_beams=2)}')
print(f'\tBeam serach (k=3): {translate(model, tokenizer, pt_text_1, "pt_PT", num_beams=3)}')
print(f'\tBeam serach (k=5): {translate(model, tokenizer, pt_text_1, "pt_PT", num_beams=5)}')
print(f'\tBeam serach (k=20): {translate(model, tokenizer, pt_text_1, "pt_PT", num_beams=20)}')
print()

print(f'Sentence #4: {pt_text_2}')
print(f'\tGreedy: {translate(model, tokenizer, pt_text_2, "pt_PT")}')
print(f'\tBeam serach (k=2): {translate(model, tokenizer, pt_text_2, "pt_PT", num_beams=2)}')
print(f'\tBeam serach (k=3): {translate(model, tokenizer, pt_text_2, "pt_PT", num_beams=3)}')
print(f'\tBeam serach (k=5): {translate(model, tokenizer, pt_text_2, "pt_PT", num_beams=5)}')
print(f'\tBeam serach (k=20): {translate(model, tokenizer, pt_text_2, "pt_PT", num_beams=20)}')

Sentence #1: Ahneus vie linnut taivaalta ja kalat merestä.
	Greedy: Greed takes birds out of heaven and fish out of the sea.
	Beam serach (k=2): Greed takes birds out of heaven and fish out of the sea.
	Beam serach (k=3): Greed takes birds out of heaven and fish out of the sea.
	Beam serach (k=5): Greed takes birds out of heaven and fish out of the sea.
	Beam serach (k=20): Greed takes birds out of heaven and fish out of the sea.

Sentence #2: Aika on rahaa, sanoi työtön kun kellonsa myi.
	Greedy: Time is money, said the unemployed when he sold his watch.
	Beam serach (k=2): Time is money, said the unemployed when he sold his watch.
	Beam serach (k=3): Time is money, said the unemployed when he sold his watch.
	Beam serach (k=5): Time is money, said the unemployed when he sold his watch.
	Beam serach (k=20): Time is money, said the unemployed when he sold his watch.

Sentence #3: Mais vale um pássaro na mão do que dois voando.
	Greedy: But it's worth a penny in the hand of the two flyi